<a href="https://colab.research.google.com/github/DuckShyamalan/COMP6248-Reproducibility-Challenge/blob/main/Distortions_and_FGSM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

     |████████████████████████████████| 143kB 6.8MB/s 


In [2]:
import torch
import torchbearer
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
import numpy as np
import pickle
from torchbearer.callbacks import LiveLossPlot
from itertools import product
import gc
from scipy.ndimage import gaussian_filter
import numpy as np
from matplotlib import pyplot as plt
import random
import torch.nn.functional as F
from torch import nn
import sys
import torchvision.utils
from torchvision import models
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [3]:
# fix random seed for reproducibility
seed = 7
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [4]:
# flatten 28*28 images to a 784 vector for each image
transform = transforms.Compose([
    transforms.ToTensor(),  # convert to tensor
    transforms.Lambda(lambda x: x.view(-1))  # flatten into vector
])

In [ ]:
#models
#Baseline
class BaselineModel(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(BaselineModel, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size) 
    self.fc2 = nn.Linear(hidden_size, hidden_size) 
    self.fc3 = nn.Linear(hidden_size, num_classes)  
    
  def forward(self, x):
      out = self.fc1(x)
      out = F.relu(out)
      out = F.dropout(out, 0.2)        
      out = self.fc2(out)        
      out = F.relu(out)
      out = self.fc3(out)
      if not self.training:
          out = F.softmax(out, dim=1)
      return out

#Defensive Distillation
class DistillationNetwork(nn.Module):
  def __init__(self):
    super(DistillationNetwork, self).__init__()
    self.conv1 = nn.Conv2d(1, 32, (5, 5), padding=0)
    self.conv2 = nn.Conv2d(32, 64, (3, 3), padding=0)
    self.fc1 = nn.Linear(64 * 5**2, 1200)
    self.fc2 = nn.Linear(1200, 10)

  def forward(self, x):
    x = torch.reshape(x, [-1, 1,28,28])
    out = self.conv1(x)
    out = F.relu(out)
    out = F.max_pool2d(out, (2,2))
    out = self.conv2(out)
    out = F.relu(out)
    out = F.max_pool2d(out, (2,2))
    out = F.dropout(out, 0.2)
    out = out.view(out.shape[0], -1)
    out = self.fc1(out)
    out = F.relu(out)
    out = self.fc2(out)
    if not self.training:
      out = F.softmax(out/50, dim=1)  #authors used T=50
    return out

#Fine-tuning


#Control ANN
